## Imports

In [43]:
import os
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.api as sm
import statsmodels.formula.api as smf
import lightgbm as lgb
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.feature_selection import RFECV
from lightgbm import LGBMRegressor

## Change file location to Data

In [44]:
head, tail = os.path.split(os.getcwd())
os.chdir(os.path.join(head,'01_Data'))

In [45]:
os.listdir()

['insurance_train.csv',
 'cleaned_test.csv',
 'cleaned_data.csv',
 'insurance_test.csv',
 'cleaned_test.pkl',
 'cleaned_data.pkl']

## Reading data

In [46]:
data = pd.read_pickle('cleaned_data.pkl')
data.head()

,ID,Start_Date_Contract,Date_Last_Renewal,Date_Next_Renewal,Date_Of_Birth,Date_Of_DL_Issuance,Issurance_Broker_Agent_Channel,Years_Associates,Total_Policies_Entity,Max_Policy_Simultaneous_Force,...,Non_Continuation_Insurance_Flag,New_License,Car_Age_Cat,Ratio_Premium_Car_Value,Power_Wt_Ratio,Customer_Loyalty,New_Bhp_Risk,Years_Driving_At_Start_Date,Young_Driver,Young_Bhp_Risk
0,34429,2017-06-01,2017-06-01,2018-06-01,1996-08-29,2016-05-31,0,1,1,1,...,1,0,Old,0.044787,0.072115,1.00,0,1,1,0
1,5552,2016-09-19,2018-09-19,2019-09-19,1992-04-30,2010-08-03,0,3,1,1,...,0,0,Standard,0.019446,0.085837,1.70,0,6,0,0
2,47700,2003-01-08,2018-01-08,2019-01-08,1972-03-23,1998-02-01,1,20,2,2,...,0,0,Old,0.015644,0.076923,8.15,0,5,0,0
3,25425,2015-10-01,2018-10-01,2019-10-01,1946-03-05,1964-08-07,0,8,2,3,...,0,0,Standard,0.010544,0.070565,4.15,0,51,0,0
4,4727,2017-01-26,2018-01-26,2019-01-26,1973-04-25,1998-07-24,0,9,1,1,...,0,0,Old,0.020250,0.047004,3.80,0,19,0,0


In [72]:
data.columns

Index(['ID', 'Start_Date_Contract', 'Date_Last_Renewal', 'Date_Next_Renewal',
       'Date_Of_Birth', 'Date_Of_DL_Issuance',
       'Issurance_Broker_Agent_Channel', 'Years_Associates',
       'Total_Policies_Entity', 'Max_Policy_Simultaneous_Force',
       'Max_Product_Simultaneous_Held', 'Policies_Terminated_Non_Payment',
       'Half_Yearly_Payment_Method', 'Premium_Amt_Current_Yr',
       'Total_Cost_Claims_Current_Yr', 'Total_Number_Claims_Current_Yr',
       'Total_Number_Claims_Entire_Duration',
       'Ratio_Claims_Total_Duration_Force',
       'Motorbikes_Vans_Cars_Agricultural', 'Rural_Urban_Flag',
       'Multiple_Drivers_Regular_Flag', 'Yr_Vehicle_Registration',
       'Vehicle_Power_HP', 'Cylinder_Capacity', 'Market_Value_EOY19',
       'Vehicle_Doors', 'Energy_Source', 'Vehicle_Wt_Kg', 'Loss_Cost',
       'Historically_Adjusted_Loss_Cost', 'Claim_Status', 'Age',
       'Years_Driving', 'Car_Age', 'Time_Since_Last_Renewal',
       'Non_Payment_Termination', 'Non_Continuati

In [73]:
X = data.copy()

In [74]:
X = X.drop(columns=['ID', 'Total_Cost_Claims_Current_Yr', 'Total_Number_Claims_Current_Yr',
                    'Total_Number_Claims_Entire_Duration', 'Ratio_Claims_Total_Duration_Force',
                    'Loss_Cost', 'Historically_Adjusted_Loss_Cost', 'Claim_Status'])
X.columns

Index(['Start_Date_Contract', 'Date_Last_Renewal', 'Date_Next_Renewal',
       'Date_Of_Birth', 'Date_Of_DL_Issuance',
       'Issurance_Broker_Agent_Channel', 'Years_Associates',
       'Total_Policies_Entity', 'Max_Policy_Simultaneous_Force',
       'Max_Product_Simultaneous_Held', 'Policies_Terminated_Non_Payment',
       'Half_Yearly_Payment_Method', 'Premium_Amt_Current_Yr',
       'Motorbikes_Vans_Cars_Agricultural', 'Rural_Urban_Flag',
       'Multiple_Drivers_Regular_Flag', 'Yr_Vehicle_Registration',
       'Vehicle_Power_HP', 'Cylinder_Capacity', 'Market_Value_EOY19',
       'Vehicle_Doors', 'Energy_Source', 'Vehicle_Wt_Kg', 'Age',
       'Years_Driving', 'Car_Age', 'Time_Since_Last_Renewal',
       'Non_Payment_Termination', 'Non_Continuation_Insurance_Flag',
       'New_License', 'Car_Age_Cat', 'Ratio_Premium_Car_Value',
       'Power_Wt_Ratio', 'Customer_Loyalty', 'New_Bhp_Risk',
       'Years_Driving_At_Start_Date', 'Young_Driver', 'Young_Bhp_Risk'],
      dtype='object')

In [75]:
Y_reg = data[['Loss_Cost','Historically_Adjusted_Loss_Cost']]
Y_class = data[['Claim_Status']]

## Fixing the X data

In [76]:
X.isna().sum()

Start_Date_Contract                    0
Date_Last_Renewal                      0
Date_Next_Renewal                      0
Date_Of_Birth                          0
Date_Of_DL_Issuance                    0
Issurance_Broker_Agent_Channel         0
Years_Associates                       0
Total_Policies_Entity                  0
Max_Policy_Simultaneous_Force          0
Max_Product_Simultaneous_Held          0
Policies_Terminated_Non_Payment        0
Half_Yearly_Payment_Method             0
Premium_Amt_Current_Yr                 0
Motorbikes_Vans_Cars_Agricultural      0
Rural_Urban_Flag                       0
Multiple_Drivers_Regular_Flag          0
Yr_Vehicle_Registration                0
Vehicle_Power_HP                       0
Cylinder_Capacity                      0
Market_Value_EOY19                     0
Vehicle_Doors                          0
Energy_Source                        593
Vehicle_Wt_Kg                          0
Age                                    0
Years_Driving   

In [77]:
data.isna().sum()[data.isna().sum() > 0]

Energy_Source                        593
Loss_Cost                          33300
Historically_Adjusted_Loss_Cost    33300
dtype: int64

In [78]:
data.loc[data['Energy_Source'].isna(),'Claim_Status'].value_counts()

Claim_Status
0    564
1     29
Name: count, dtype: int64

We can fill the X['Energy_Source'] with other

In [79]:
X['Energy_Source'] = X['Energy_Source'].fillna('Other')
X['Energy_Source'].value_counts(dropna = False)

Energy_Source
D        23074
P        13784
Other      593
Name: count, dtype: int64

Create categorical variables

In [80]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37451 entries, 0 to 37450
Data columns (total 38 columns):
 #   Column                             Non-Null Count  Dtype         
---  ------                             --------------  -----         
 0   Start_Date_Contract                37451 non-null  datetime64[ns]
 1   Date_Last_Renewal                  37451 non-null  datetime64[ns]
 2   Date_Next_Renewal                  37451 non-null  datetime64[ns]
 3   Date_Of_Birth                      37451 non-null  datetime64[ns]
 4   Date_Of_DL_Issuance                37451 non-null  datetime64[ns]
 5   Issurance_Broker_Agent_Channel     37451 non-null  int64         
 6   Years_Associates                   37451 non-null  int64         
 7   Total_Policies_Entity              37451 non-null  int64         
 8   Max_Policy_Simultaneous_Force      37451 non-null  int64         
 9   Max_Product_Simultaneous_Held      37451 non-null  int64         
 10  Policies_Terminated_Non_Payment   

In [81]:
X = pd.get_dummies(X, columns=['Car_Age_Cat', 'Energy_Source'], dtype=int, drop_first=False)
X.head()

,Start_Date_Contract,Date_Last_Renewal,Date_Next_Renewal,Date_Of_Birth,Date_Of_DL_Issuance,Issurance_Broker_Agent_Channel,Years_Associates,Total_Policies_Entity,Max_Policy_Simultaneous_Force,Max_Product_Simultaneous_Held,...,Years_Driving_At_Start_Date,Young_Driver,Young_Bhp_Risk,Car_Age_Cat_New,Car_Age_Cat_Recent,Car_Age_Cat_Standard,Car_Age_Cat_Old,Energy_Source_D,Energy_Source_Other,Energy_Source_P
0,2017-06-01,2017-06-01,2018-06-01,1996-08-29,2016-05-31,0,1,1,1,1,...,1,1,0,0,0,0,1,0,0,1
1,2016-09-19,2018-09-19,2019-09-19,1992-04-30,2010-08-03,0,3,1,1,1,...,6,0,0,0,0,1,0,1,0,0
2,2003-01-08,2018-01-08,2019-01-08,1972-03-23,1998-02-01,1,20,2,2,1,...,5,0,0,0,0,0,1,0,0,1
3,2015-10-01,2018-10-01,2019-10-01,1946-03-05,1964-08-07,0,8,2,3,1,...,51,0,0,0,0,1,0,1,0,0
4,2017-01-26,2018-01-26,2019-01-26,1973-04-25,1998-07-24,0,9,1,1,1,...,19,0,0,0,0,0,1,0,0,1


In [82]:
X[['Car_Age_Cat_New','Car_Age_Cat_Recent','Car_Age_Cat_Standard','Car_Age_Cat_Old']].sum()

Car_Age_Cat_New          2159
Car_Age_Cat_Recent       3402
Car_Age_Cat_Standard    16978
Car_Age_Cat_Old         14912
dtype: int64

In [86]:
X = X.drop(columns=['Car_Age_Cat_New', 'Energy_Source_Other'])

Now we need to drop time

In [87]:
X = X.drop(columns=['Start_Date_Contract','Date_Last_Renewal','Date_Next_Renewal','Date_Of_Birth','Date_Of_DL_Issuance'])

## Fixing Y

In [98]:
Y_reg = Y_reg.fillna(0)

In [99]:
Y_reg.value_counts()

Loss_Cost   Historically_Adjusted_Loss_Cost
0.000       0.00000                            33300
882.000     882.00000                             40
            1764.00000                            10
            1746.36000                             9
            1755.18000                             8
                                               ...  
96.150      192.30000                              1
96.300      135.78300                              1
96.335      230.24065                              1
96.410      157.14830                              1
118142.590  236285.18000                           1
Name: count, Length: 3906, dtype: int64

## Now we need to split the data appropriately

We need to stratify it while splitting

In [91]:
X_class_train, X_class_test, Y_class_train, Y_class_test = train_test_split(
    X, Y_class,
    test_size=0.2,
    stratify=Y_class,
    random_state=42
)

print(X_class_test.shape)
print(X_class_train.shape)
print(Y_class_test.shape)
print(Y_class_train.shape)

(7491, 36)
(29960, 36)
(7491, 1)
(29960, 1)


In [93]:
display(Y_class_train.value_counts(normalize=True))
display(Y_class_test.value_counts(normalize=True))

Claim_Status
0               0.889152
1               0.110848
Name: proportion, dtype: float64

Claim_Status
0               0.8892
1               0.1108
Name: proportion, dtype: float64

For regressors we need to mention 0 before stratify

In [100]:
# using Loss Cost as HALC might have some 0
zero_flag = (Y_reg['Loss_Cost'] == 0).astype(int)
zero_flag.value_counts(dropna=False)

Loss_Cost
1    33300
0     4151
Name: count, dtype: int64

In [101]:
X_reg_train, X_reg_test, Y_reg_train, Y_reg_test = train_test_split(
    X, Y_reg,
    test_size=0.2,
    stratify=zero_flag,
    random_state=42
)

In [102]:
print(X_reg_test.shape)
print(X_reg_train.shape)
print(Y_reg_test.shape)
print(Y_reg_train.shape)

(7491, 36)
(29960, 36)
(7491, 2)
(29960, 2)


In [104]:
display(len(Y_reg_train[Y_reg_train['Loss_Cost']==0])/len(Y_reg_train))
display(len(Y_reg_test[Y_reg_test['Loss_Cost']==0])/len(Y_reg_test))

0.8891522029372496

0.8892003737818716